In [ ]:
import math
import os
import shutil

import dask.dataframe as dd
import numpy as np
import pandas as pd
from google.colab import drive
from matplotlib import pyplot
from numpy import pi

path = os.getcwd()+"/drive/MyDrive/"

### PREPROCESSING DATA

In [ ]:
drive.mount('/content/drive')

In [ ]:
oldDirectory = path+"KFall/sensor_data"
newDirectory = path+"Processed/raw_data"

fileNames = os.listdir(oldDirectory)

for fn in fileNames:
    oldDir = os.path.join(oldDirectory, fn)
    newDir = os.path.join(newDirectory, fn)
    shutil.copytree(oldDir, newDir)

print(len(fileNames))

In [ ]:
directory = path+"KFall/label_data"
rawDataDirectory = path+"Processed/raw_data"
newDirInput = path+"Processed/input_label_data"

files = os.listdir(directory)

for f in files:
    subject = f.split()[0][:4]
    subject_id = f.split()[0][2:4]
    if int(subject_id) < 20:
        data = pd.read_excel(os.path.join(directory, f),
                             skiprows=1, header=None)
        rawDirectory = rawDataDirectory + "/" + subject + "/"
        for i in range(len(data)):
            activity = str(data.loc[i, 0])
            if activity[0] != 'nan':
                activity_id = activity[5:7]
            trial = data.loc[i, 2]
            frame_onset = data.loc[i, 3]
            frame_impact = data.loc[i, 4]
            filename = "S"+subject_id+"T"+activity_id+"R0"+str(trial)+".csv"
            selected_files = os.listdir(rawDirectory)
            for file in selected_files:
                act_id = int(file.split('.')[0][4:6])
                tr_id = int(file.split('.')[0][-2:])
                exist = os.path.exists(os.path.join(newDirInput, file))
                if int(act_id) > 19 and int(act_id) < 35:
                    df = pd.read_csv(os.path.join(rawDirectory, file))
                    df["Condition"] = 0
                    if file == filename:
                        for ind in range(len(df)):
                            if ind <= frame_onset:
                                df.loc[ind, "Condition"] = 0
                            elif ind > frame_onset and ind < frame_impact:
                                df.loc[ind, "Condition"] = 1
                            elif ind >= frame_impact:
                                df.loc[ind, "Condition"] = 2

                        df.to_csv(os.path.join(newDirInput, file),
                                  sep=",", header=False, index=False)
                        print(file, "saved to", newDirInput, "!")

                elif int(act_id) == 17 and exist is False:
                    print(file)
                    df = pd.read_csv(os.path.join(rawDirectory, file))
                    df["Condition"] = 2

    df.to_csv(os.path.join(newDirInput, file),
              sep=",", header=False, index=False)
    print(file, "saved to", newDirInput, "!")

In [ ]:
directory = path+"Processed/input_label_data"
new_directory = os.getcwd()+"/drive/MyDrive/Processed/processed_input_label_data/"

raw_files = os.listdir(directory)

for f in raw_files:
    dataframe = pd.read_csv(os.path.join(directory, f),
                            header=None, usecols=[0, 2, 3, 4, 5, 6, 7, 11])

    dataframe["C1"] = 0.0
    dataframe["C2"] = 0.0
    dataframe["Acc_roll"] = 0.0
    dataframe["Acc_pitch"] = 0.0
    dataframe["Gyro_roll"] = 0.0
    dataframe["Gyro_pitch"] = 0.0
    dataframe["Roll"] = 0.0
    dataframe["Pitch"] = 0.0
    dataframe["Yaw"] = 0.0

    dataframe.to_csv(os.path.join(new_directory, f),
                     sep=",", header=False, index=False)
    print(os.path.join(new_directory, f), ' saved!')

In [ ]:
directory = os.getcwd()+"/drive/MyDrive/Processed/processed_input_label_data/"

raw_files = os.listdir(directory)

n_freq = 100
d_time = 1/n_freq

rad_to_deg = 180/np.pi

const_time = 2
comp_const = const_time/(const_time+d_time)

for f in raw_files:
    dataframe = pd.read_csv(os.path.join(directory, f), header=None, names=[
                            "Timestamp", "Ax", "Ay", "Az", "Gx", "Gy", "Gz", "Condition", "C1", "C2", "Acc_roll", "Acc_pitch", "Gyro_roll", "Gyro_pitch", "Roll", "Pitch", "Yaw"])

    for ind in range(len(dataframe)):
        d_time = 0
        if ind == 0:
            d_time = dataframe.loc[ind, "Timestamp"] - 0
        else:
            d_time = float(
                dataframe.loc[ind, "Timestamp"] - dataframe.loc[(ind-1), "Timestamp"])

        dataframe.loc[ind, "C1"] = np.sqrt(
            np.power(dataframe.loc[ind, "Ax"], 2) + np.power(dataframe.loc[ind, "Az"], 2))

        dataframe.loc[ind, "C2"] = np.sqrt(np.power(dataframe.loc[ind, "Ax"], 2) + np.power(
            dataframe.loc[ind, "Ay"], 2) + np.power(dataframe.loc[ind, "Az"], 2))
        dataframe.loc[ind, "Acc_roll"] = math.atan2(
            - (dataframe.loc[ind, "Az"]), dataframe.loc[ind, "Ay"])*rad_to_deg
        dataframe.loc[ind, "Acc_pitch"] = math.atan2(dataframe.loc[ind, "Ax"], (np.sqrt(
            np.power(dataframe.loc[ind, "Az"], 2) + np.power(dataframe.loc[ind, "Ay"], 2))))*rad_to_deg
        dataframe.loc[ind, "Gyro_roll"] = dataframe.loc[ind,
                                                        "Gyro_roll"] + (dataframe.loc[ind, "Gz"]*d_time)
        dataframe.loc[ind, "Gyro_pitch"] = dataframe.loc[ind,
                                                         "Gyro_pitch"] + (dataframe.loc[ind, "Gx"]*d_time)
        dataframe.loc[ind, "Roll"] = dataframe.loc[ind, "Roll"] + \
            (dataframe.loc[ind, "Gz"]*d_time)*comp_const + \
            dataframe.loc[ind, "Acc_roll"]*(1-comp_const)
        dataframe.loc[ind, "Pitch"] = dataframe.loc[ind, "Pitch"] + \
            (dataframe.loc[ind, "Gx"]*d_time)*comp_const + \
            dataframe.loc[ind, "Acc_pitch"]*(1-comp_const)
        dataframe.loc[ind, "Yaw"] = float(
            dataframe.loc[ind, "Yaw"] + (dataframe.loc[ind, "Gy"]*d_time))

        if dataframe.loc[ind, "Yaw"] < 0:
            dataframe.loc[ind, "Yaw"] = float(dataframe.loc[ind, "Yaw"]+360.0)

    dataframe.to_csv(os.path.join(directory, f),
                     sep=",", header=False, index=False)
    print(f, ' saved!')

In [ ]:
pathfile = path+"Processed/processed_input_label_data/*csv"

df = dd.read_csv(pathfile, header=None, usecols=[
                 0, 1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 15, 16])

df.columns = ["Timestamp", "Ax", "Ay", "Az", "Gx", "Gy",
              "Gz", "Condition", "C1", "C2", "Roll", "Pitch", "Yaw"]

new_cols = ["Timestamp", "Ax", "Ay", "Az", "Gx", "Gy",
            "Gz", "C1", "C2", "Roll", "Pitch", "Yaw", "Condition"]
df = df[new_cols]

new_directory = path+"/Processed/"

single_filename = "single_file_kfalldataset_3labels.csv"
print(path, " read!")

df.to_csv(os.path.join(new_directory, single_filename),
          index=False, single_file=True)

print(os.path.join(new_directory, single_filename), " saved!")